In [1]:
import pickle
import gensim
from nltk.corpus import stopwords 

In [2]:
f = open("../Dataset/BOW.txt", "r")

for line in f:
    list_of_words = line.split(',')
f.close()

index_to_words = {}
for i in range(len(list_of_words)):
    index_to_words[i+1] = list_of_words[i]

In [3]:
f = open("../Dataset/dataset.pkl", "rb")
msdid_to_lyrics =  pickle.load(f)
f.close()

msd_ids = list(msdid_to_lyrics.keys())

In [4]:
stop_words = []
languages_list = ['english', 'spanish', 'german', 'french', 'italian']
for language in languages_list:
    stop_words.extend(stopwords.words(language))

from nltk.stem import PorterStemmer 
ps = PorterStemmer() 

stop_words = [ps.stem(w) for w in stop_words]
# print(stop_words)


In [5]:
processed_docs = []

for msd_id in msd_ids:
    lyrics = msdid_to_lyrics[msd_id][-1]

    word_lyrics = []
    for c in lyrics:
        collon_index = c.find(':')
        ind = int(c[:collon_index])
        freq = int(c[collon_index+1:])
        for i in range(freq):
            w = index_to_words[ind]
            if len(w)<=3 or w in stop_words:
                continue
            word_lyrics.append(w)
    processed_docs.append(word_lyrics)

In [6]:
print(len(processed_docs))

237662


In [7]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [8]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 arrang
1 captur
2 damn
3 devast
4 element
5 fashion
6 flesh
7 grace
8 grant
9 highest
10 ignor


In [9]:
dictionary.filter_extremes(no_below=15, no_above=0.5)

In [10]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [11]:
lda_model = gensim.models.LdaModel(bow_corpus, num_topics=20, id2word=dictionary, passes=2, alpha = 'auto')

In [12]:
lda_model.save("lda_Model_topics_20")

In [13]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5724731936938867


In [15]:
print(lda_model.print_topics())


[(0, '0.054*"solo" + 0.036*"piÃ¹" + 0.034*"quando" + 0.022*"sempr" + 0.022*"senza" + 0.020*"tempo" + 0.018*"ancora" + 0.017*"dentro" + 0.017*"cosa" + 0.017*"vita"'), (1, '0.089*"world" + 0.058*"sing" + 0.054*"song" + 0.051*"live" + 0.041*"peopl" + 0.028*"togeth" + 0.026*"bring" + 0.025*"hear" + 0.024*"shine" + 0.024*"stand"'), (2, '0.074*"time" + 0.054*"feel" + 0.050*"wanna" + 0.049*"girl" + 0.048*"know" + 0.042*"make" + 0.041*"take" + 0.037*"right" + 0.032*"good" + 0.031*"tell"'), (3, '0.055*"littl" + 0.029*"ride" + 0.022*"star" + 0.022*"water" + 0.018*"like" + 0.015*"sweet" + 0.014*"wild" + 0.013*"river" + 0.012*"white" + 0.012*"child"'), (4, '0.211*"heart" + 0.082*"break" + 0.041*"beat" + 0.033*"anoth" + 0.033*"apart" + 0.031*"tear" + 0.030*"broken" + 0.016*"part" + 0.012*"woah" + 0.011*"toujour"'), (5, '0.039*"like" + 0.017*"this" + 0.017*"nigga" + 0.014*"shit" + 0.013*"fuck" + 0.013*"rock" + 0.011*"back" + 0.011*"caus" + 0.009*"money" + 0.008*"roll"'), (6, '0.042*"know" + 0.038*"w

In [18]:
res = lda_model.get_document_topics(bow_corpus)

In [20]:
topic_count = []
for i in range(20):
    topic_count.append(0)
list1 = []
for i in range(len(res)):
    a = []
    for c in res[i]:
        if c[1] >0.24999:
            a.append(c[0])
    list1.append(a)
    for j in range(20):
        if j in a:
            topic_count[j] += 1
            
    if(i%10000 == 0):
        print(i)
print(topic_count)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
[4867, 3639, 15470, 5433, 1547, 20110, 67794, 2843, 7068, 7120, 5282, 10404, 1953, 1819, 53261, 18932, 4749, 5945, 27970, 15589]


In [21]:
lda_model_2 = gensim.models.LdaMulticore(bow_corpus, num_topics=40, id2word=dictionary, passes=2, workers = 2)

In [22]:
lda_model_2.save("lda_Model_topics_40")

In [23]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model_2, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.46624006363405435


In [24]:
print(lda_model_2.print_topics())

[(38, '0.094*"call" + 0.061*"better" + 0.053*"chorus" + 0.046*"name" + 0.029*"readi" + 0.029*"hear" + 0.021*"hous" + 0.021*"brother" + 0.018*"line" + 0.016*"number"'), (13, '0.018*"soul" + 0.018*"blood" + 0.017*"burn" + 0.016*"kill" + 0.016*"death" + 0.011*"fire" + 0.010*"this" + 0.009*"dead" + 0.008*"rise" + 0.008*"hell"'), (0, '0.063*"cool" + 0.039*"shut" + 0.032*"niet" + 0.027*"anim" + 0.026*"vain" + 0.025*"fever" + 0.025*"rockin" + 0.025*"maar" + 0.024*"minut" + 0.023*"monkey"'), (7, '0.180*"night" + 0.126*"danc" + 0.111*"fall" + 0.028*"christma" + 0.028*"ladi" + 0.022*"send" + 0.020*"white" + 0.012*"floor" + 0.011*"dream" + 0.009*"miracl"'), (39, '0.088*"look" + 0.078*"around" + 0.060*"everi" + 0.045*"break" + 0.026*"breath" + 0.025*"make" + 0.022*"turn" + 0.017*"enough" + 0.016*"ground" + 0.014*"noth"'), (30, '0.588*"yeah" + 0.151*"wait" + 0.033*"beat" + 0.013*"rhythm" + 0.008*"loos" + 0.007*"long" + 0.006*"time" + 0.006*"kick" + 0.006*"ohoh" + 0.005*"dancin"'), (15, '0.207*"beli

In [25]:
lda_model_3 = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=2, workers = 2)

In [26]:
lda_model_3.save("lda_Model_topics_20_alpha_not_def")

In [27]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model_3, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5868904872668171


In [28]:
print(lda_model_3.print_topics())

[(0, '0.075*"danc" + 0.040*"fire" + 0.028*"citi" + 0.025*"black" + 0.024*"burn" + 0.024*"move" + 0.022*"parti" + 0.019*"white" + 0.017*"water" + 0.015*"town"'), (1, '0.034*"like" + 0.018*"nigga" + 0.017*"fuck" + 0.017*"this" + 0.015*"shit" + 0.011*"caus" + 0.009*"money" + 0.009*"back" + 0.009*"bitch" + 0.009*"know"'), (2, '0.153*"gonna" + 0.119*"wanna" + 0.063*"give" + 0.063*"make" + 0.053*"gotta" + 0.025*"ride" + 0.024*"turn" + 0.023*"round" + 0.018*"take" + 0.017*"around"'), (3, '0.026*"would" + 0.016*"could" + 0.015*"home" + 0.013*"said" + 0.012*"like" + 0.012*"time" + 0.011*"this" + 0.011*"never" + 0.009*"night" + 0.009*"look"'), (4, '0.030*"amor" + 0.021*"quiero" + 0.019*"vida" + 0.015*"mÃ¡s" + 0.011*"solo" + 0.011*"siempr" + 0.010*"nunca" + 0.009*"tiempo" + 0.008*"corazÃ³n" + 0.008*"ahora"'), (5, '0.044*"cest" + 0.033*"tout" + 0.028*"plus" + 0.026*"comm" + 0.020*"nous" + 0.017*"quand" + 0.014*"fait" + 0.014*"bien" + 0.013*"refrain" + 0.013*"rien"'), (6, '0.019*"light" + 0.013*"da

In [29]:
res2 = lda_model_3.get_document_topics(bow_corpus)

In [30]:
topic_count = []
for i in range(20):
    topic_count.append(0)
list1 = []
for i in range(len(res2)):
    a = []
    for c in res2[i]:
        if c[1] >0.24999:
            a.append(c[0])
    list1.append(a)
    for j in range(20):
        if j in a:
            topic_count[j] += 1
            
    if(i%10000 == 0):
        print(i)
print(topic_count)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
[3927, 17922, 4574, 46871, 15305, 6638, 31599, 3862, 5480, 13394, 5103, 8769, 506, 34511, 9236, 26340, 10071, 10432, 29265, 33937]
